Modèle sans convolution (réseau de neurones classique) sur le dataset du Joueur bas - Henri
- Coup droit
- Revers

In [93]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder

# Coup droit et revers

## Prises des frames et des vidéos

In [94]:
list_videos = []

In [95]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk('img/JBas'):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [96]:
list_videos

In [97]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers']

In [98]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: revers

In [99]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int64)

In [100]:
# Résolution des images
list_videos[0].frames[0].shape

(50, 50)

In [101]:
len(list_videos)

67

In [102]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [103]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(53, 53, 14, 14)

In [104]:
paths_test

['img/JBas\\coup droit\\car0001-0342253\\images',
 'img/JBas\\coup droit\\car5493-5847185\\images',
 'img/JBas\\coup droit\\na02011927\\images',
 'img/JBas\\coup droit\\na02012021\\images',
 'img/JBas\\coup droit\\na0201737\\images',
 'img/JBas\\coup droit\\uopc1606\\images',
 'img/JBas\\coup droit\\uopc424\\images',
 'img/JBas\\coup droit\\uopc698\\images',
 'img/JBas\\coup droit\\uopc955\\images',
 'img/JBas\\revers\\a36r977\\images',
 'img/JBas\\revers\\na02011122\\images',
 'img/JBas\\revers\\na02011195\\images',
 'img/JBas\\revers\\na0201493\\images',
 'img/JBas\\revers\\uopc1371\\images']

In [105]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1])

In [106]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [107]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Modèle de réseau de neurones avec classification binaires

In [124]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = 50*50*15), # 50x50x15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=128, activation="linear"),
        Dense(units=1, activation="sigmoid"),
    ]
)          
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 16)                600016    
                                                                 
 dense_38 (Dense)            (None, 32)                544       
                                                                 
 dense_39 (Dense)            (None, 128)               4224      
                                                                 
 dense_40 (Dense)            (None, 1)                 129       
                                                                 
Total params: 604,913
Trainable params: 604,913
Non-trainable params: 0
_________________________________________________________________


In [125]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [130]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs = 100,
    # batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/100
1/2 [==============>...............] - ETA: 0s - loss: 3.0558e-07 - accuracy: 1.0000
Epoch 1: val_loss did not improve from 0.00000
2/2 [==============================] - 1s 99ms/step - loss: 4.0319 - accuracy: 0.9524 - val_loss: 16.6327 - val_accuracy: 0.6364
Epoch 2/100
1/2 [==============>...............] - ETA: 0s - loss: 2.7000e-06 - accuracy: 1.0000
Epoch 2: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 28ms/step - loss: 8.3102 - accuracy: 0.8571 - val_loss: 10.2644 - val_accuracy: 0.7273
Epoch 3/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0465 - accuracy: 0.9688
Epoch 3: val_loss did not improve from 0.00000
2/2 [==============================] - 0s 49ms/step - loss: 0.0354 - accuracy: 0.9762 - val_loss: 221.1434 - val_accuracy: 0.0000e+00
Epoch 4/100
1/2 [==============>...............] - ETA: 0s - loss: 15.3073 - accuracy: 0.8125
Epoch 4: val_loss did not improve from 0.00000
2/2 [==============================] - 

In [131]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 16ms/step - loss: 84.7713 - accuracy: 0.2857


[84.77130889892578, 0.2857142984867096]

In [132]:
pred = model.predict(X_test)
prediction_p = tf.nn.sigmoid(pred)
prediction_p

1/1 [==============================] - 0s 36ms/step


<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[0.7310586 ],
       [0.7310586 ],
       [0.7310586 ],
       [0.7310586 ],
       [0.7310586 ],
       [0.5       ],
       [0.7310586 ],
       [0.7310586 ],
       [0.7310586 ],
       [0.5       ],
       [0.7310586 ],
       [0.50000006],
       [0.7310586 ],
       [0.7310586 ]], dtype=float32)>

In [133]:
y_pred = tf.where(prediction_p < 0.5, 0, 1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  tf.Tensor(
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]], shape=(14, 1), dtype=int32)
True labels:       [0 0 0 0 0 0 0 0 0 1 1 1 1 1]


**Accuracy: $5/14$ ($35.71\%$)**

# Coup droit, revers, service et déplacement

## Prises des frames et des vidéos

In [39]:
list_videos = []
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk('img/JBas'):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [40]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacem

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [42]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
      dtype=int64)

In [43]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [44]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [45]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(104, 104, 26, 26)

In [46]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)

In [47]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [48]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Modèle de réseau de neurones avec classification multicritères

In [65]:
model_2 = Sequential(
    [               
        tf.keras.Input(shape = 50*50*15), # 50*50 * 15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="linear"),
    ]
)          
model_2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 16)                600016    
                                                                 
 dense_21 (Dense)            (None, 32)                544       
                                                                 
 dense_22 (Dense)            (None, 64)                2112      
                                                                 
 dense_23 (Dense)            (None, 128)               8320      
                                                                 
 dense_24 (Dense)            (None, 4)                 516       
                                                                 
Total params: 611,508
Trainable params: 611,508
Non-trainable params: 0
_________________________________________________________________


In [66]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [69]:
model_2.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer = Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history_2 = model_2.fit(
    X_train, y_train,
    epochs = 100,
    # batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/100
1/3 [=========>....................] - ETA: 0s - loss: 1.0581 - accuracy: 0.4375
Epoch 1: val_loss did not improve from 1.78010
3/3 [==============================] - 0s 55ms/step - loss: 1.0325 - accuracy: 0.4940 - val_loss: 6.1551 - val_accuracy: 0.0000e+00
Epoch 2/100
1/3 [=========>....................] - ETA: 0s - loss: 0.9922 - accuracy: 0.5312
Epoch 2: val_loss did not improve from 1.78010
3/3 [==============================] - 0s 16ms/step - loss: 1.0261 - accuracy: 0.4940 - val_loss: 6.2780 - val_accuracy: 0.0000e+00
Epoch 3/100
1/3 [=========>....................] - ETA: 0s - loss: 0.9818 - accuracy: 0.5312
Epoch 3: val_loss did not improve from 1.78010
3/3 [==============================] - 0s 15ms/step - loss: 1.0288 - accuracy: 0.4940 - val_loss: 6.3028 - val_accuracy: 0.0000e+00
Epoch 4/100
1/3 [=========>....................] - ETA: 0s - loss: 1.0126 - accuracy: 0.5000
Epoch 4: val_loss did not improve from 1.78010
3/3 [==============================] - 0s 17

In [71]:
model_2.evaluate(X_test, y_test)

1/1 [==============================] - 0s 18ms/step - loss: 1.5369 - accuracy: 0.5000


[1.5369466543197632, 0.5]

In [72]:
pred = model_2.predict(X_test)
pred

1/1 [==============================] - 0s 41ms/step


array([[  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1.7456586 ,   0.9571542 , -12.253982  ],
       [  1.2368557 ,   1

In [73]:
prediction = tf.nn.softmax(pred)
prediction

<tf.Tensor: shape=(26, 4), dtype=float32, numpy=
array([[2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4.0463667e-07],
       [2.9245669e-01, 4.8644286e-01, 2.2109999e-01, 4

In [74]:
y_pred = np.argmax(prediction, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True labels:       [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 3]
